## Аггрегация разметки датасета ruMultiAr

Аггрегация строится по следующей системе:

1. Сбор размеченных пулов с Толоки. Возможны варианты:
    - только общий пул нужно аггрегировать, тогда забирается только он
    - часть данных находится в контрольных заданиях и экзамене, тогда к основному пулу добавляются данные задания
2. Фильтрация разметчиков:
    - в общем пуле есть некоторое количество заранее размеченных заданий - контрольных
    - хорошим считается разметчик, который показывает `accuracy >= 0.5` на данных заданиях
    - формируется список "плохих" разметчиков
3. Аггрегация ответов разметчиков по заданиям:
    - форматирование в заданиях может отличаться от изначального из-за выгрузки с Толоки
    - учитываются только ответы "хороших" разметчиков
    - аггрегация по подготовленным пулам - создается массив карточек вида {key: value}, где key - кортеж из всех значимых элементов задания, value - список из кортежей вида (user_id, answer)
4. Голосование большинством по каждому заданию:
    - минимально необходимое большинство составляет 3 голоса, так как такое большинство валидно для перекрытия 5
    - по результату формируется датафрейм с заданиями и ответами
5. Подгрузка оригинальных данных с разметкой в виде таблицы с заданиями и ответами
6. Соединение таблиц:
    - очистка форматирования в таблице с ответами разметчиков и в таблице с правильными ответами
    - создание единых столбцов с полным заданием
    - соединение таблиц по данному столбцу
    - валидация размеров
7. Подсчет метрик

In [1]:
import pandas as pd
from collections import Counter
import numpy as np

### Сбор данных разметки и фильтрация разметчиков

Данный датасет представляет из себя лишь подмножество, которое было специально сгенерировано. Это не тестовый сет датасета из мультимодального бенчмарка (инструкционный сет)! Размер выборки - 600 объектов.

In [2]:
assignments = pd.read_csv('assignments_from_pool_41562232__04-10-2023.tsv', sep='\t')
skills = pd.read_csv('workerSkills.csv', sep='|')

Разметчикам предлагалось решить математический пример.

Вход: 
- INPUT:formula (пример: `(2 + (-1))`).

Выход:
- OUTPUT:result (целое число, например: `1`).

In [3]:
assignments.head(1)

,INPUT:formula,OUTPUT:result,GOLDEN:result,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted,ASSIGNMENT:reward
0,((-6) + 9 * (-4) - (-8)),-34,NaN,NaN,NaN,https://platform.toloka.ai/task/41562232/00027...,00027a3078--6516b2ee056db464d2cd5f04,00027a3078--6516c2158879a44e701a88f7,00027a3078--6516c2158879a44e701a88e9,14098124e4a687d4023347e387bd282c,APPROVED,2023-09-29T12:24:53.371,2023-09-29T12:26:40.733,2023-09-29T12:26:40.733,0.022


Фильтруем толокеров с `accuracy < 0.5` на контрольных заданиях, чтобы не учитывать их ответы при подсчете метрик.

In [4]:
from collections import defaultdict

users_dict = defaultdict(lambda: defaultdict(int))

for idx, row in assignments.iterrows():
    formula = row[0]

    out = row[1]
    
    gold = row[2]

    user = row[9]

    if str(user) != "nan" and str(gold) != "nan":
        if out == int(gold):
            users_dict[user]["good"] += 1
        else:
            users_dict[user]["bad"] += 1

print("Users total: ", len(users_dict))
bad_users = []
for key, value in users_dict.items():
    percentage_good = value["good"]/(value["good"] + value["bad"])
    if percentage_good < 0.5:
        bad_users.append(key)

print("Bad users:", len(bad_users))

Users total:  103
Bad users: 9


9 из 103 разметчиков на контрольных заданиях показали слишком плохое качество, чтобы учитывать их ответы для расчета метрики.

Отделяем контроль от основы, так как контрольные задания создавались отдельно и не должны учитываться при подсчете метрик. На контрольных заданиях есть `GOLDEN:result`. Также отсеиваем возможные баги Толоки, когда в строке может не быть задания - `INPUT:formula` содержит NaN.

In [5]:
assignments_no_control = assignments[assignments['GOLDEN:result'].isnull()]
assignments_no_control_no_null = assignments_no_control[assignments_no_control['INPUT:formula'].notnull()]

Посчитаем, сколько было затрачено на получение разметки тестовых данных без учета контрольных заданий, так как они могли проходиться неограниченное количество раз одним и тем же разметчиком.

In [6]:
def w_sum(df):
    idx = df.index.values
    vals = df.values
    summ = idx * vals
    return summ.sum()

d1 = assignments_no_control_no_null['ASSIGNMENT:reward'].value_counts(normalize=True)
d2 = assignments_no_control_no_null['ASSIGNMENT:reward'].value_counts()
print(f'взвешенная цена айтема в тесте: {round(w_sum(d1), 3)}')
print(f'потрачено на разметку теста: {round(w_sum(d2), 3)}')
print(f'{round(w_sum(d2), 3)} / {round(w_sum(d1), 3)}')

взвешенная цена айтема в тесте: 0.025
потрачено на разметку теста: 75.942
75.942 / 0.025


Выделим, сколько составила средняя часовая ставка для разметки тестовой части датасета. Это будет простое среднее из следующих величин: количество заданий, которое разметчк может сделать за час на основе данного задания, помноженное на цену задания.

In [7]:
def get_hour_pay(df):
    try:
        times = pd.to_datetime(df['ASSIGNMENT:submitted']) - pd.to_datetime(df['ASSIGNMENT:started'])
    except Exception as e:
        times = []
        for i in range(len(df)):
            try:
                start = pd.to_datetime(df['ASSIGNMENT:started'].iloc[i])
            except Exception as e:
                start = pd.to_datetime(df['ASSIGNMENT:started'].apply(lambda x: x.split('T')[1]).iloc[i])
            try:
                end = pd.to_datetime(df['ASSIGNMENT:submitted'].iloc[i])
            except Exception as e:
                start = pd.to_datetime(df['ASSIGNMENT:submitted'].apply(lambda x: x.split('T')[1]).iloc[i])
            delta = end - start
            times.extend([delta])
        times = pd.Series(times)
        # times = pd.to_datetime(df['ASSIGNMENT:submitted'].apply(lambda x: x.split('T')[1])) - pd.to_datetime(df['ASSIGNMENT:started'].apply(lambda x: x.split('T')[1]))
    sums = 3600 / times.apply(lambda x: x.seconds) * df['ASSIGNMENT:reward']
    return sums.mean()

get_hour_pay(assignments_no_control_no_null)

1.0060694562635302

### Сбор ответов разметчиков и голосование

Собираем ответы голосования большинством для каждого задания.

In [8]:
from collections import defaultdict

text_dict = defaultdict(list)

for formula, user, out in zip(
    assignments_no_control_no_null["INPUT:formula"], assignments_no_control_no_null["ASSIGNMENT:worker_id"], 
    assignments_no_control_no_null["OUTPUT:result"]
    ):
    if user not in bad_users:
        text_dict[(formula)].append([
                user,
                {"out": out}
        ])

print(len(text_dict))

600


In [9]:
keys = list(text_dict.keys())
Counter([len(text_dict[keys[i]]) for i in range(len(keys))])

Counter({5: 562, 4: 38})

Есть 38 заданий с перектрытием 4. Для формирования итоговых лейблов нужно, чтобы было простое большинство разметчиков, проголосовавших за данную опцию. Если большинства нет, то оценка строится, исходя из оценки навыков разметчиков. В таком случае, финальный лейбл будет присвоен по голосу группы с наилучшими навыками. Если по навыкам будет равенство, то решаем по ответам топ-3 по навыкам разметчиков. Если и данный способ дает равенство, то используются оценки навыков разметчиков из EM-алгоритма (реализация GLAD).

In [10]:
preds_full = {}
user2skill = {k:v for k, v in zip(skills['worker_id'], skills['skill_value'])}
control_acc = assignments[assignments['GOLDEN:result'].notna()]\
    .groupby('ASSIGNMENT:worker_id')\
        .apply(lambda x: (np.array(x['OUTPUT:result']) == np.array(x['GOLDEN:result'])).mean())
user2control = {k:v for k, v in zip(control_acc.index, control_acc.values)}

from crowdkit.aggregation.classification.glad import GLAD

full = assignments['INPUT:formula']
id2task = dict(enumerate(full))
task2id = {k:v for v, k in id2task.items()}
id2user = dict(enumerate(assignments['ASSIGNMENT:worker_id']))
user2id = {k:v for v, k in id2user.items()}

codes = full.map(task2id)
res = pd.DataFrame({'task': codes, 'worker': assignments['ASSIGNMENT:worker_id'].map(user2id), 'label': assignments['OUTPUT:result']})
model = GLAD(n_iter=1000, tol=1e-03, m_step_max_iter=1000, m_step_tol=1e-03)
model.fit(res)
user2alpha = dict(enumerate(model.alphas_))
tb = model.alphas_.copy()
tb.index = tb.index.map(id2user)
user2alpha = {k:v for k, v in zip(tb.index, tb.values)}

stats = {
    'total_agreement': 0,
    'majority': 0,
    'skill_based': 0,
    'major_based': 0,
    'em_based': 0,
    'rest': 0,
}

for i in range(len(keys)):
    ans = text_dict[keys[i]]
    lst = [[ans[j][0], ans[j][1]['out']] for j in range(len(ans))]
    users, votes = list(zip(*lst))
    cnt = pd.Series(Counter(votes)).sort_values(ascending=False)

    # # total agreement
    if len(cnt) == 1:
        res = cnt.index[0]
        stats['total_agreement'] += 1
    # simple majority
    elif cnt.iloc[0] > cnt.iloc[1]:
        res = cnt.index[0]
        stats['majority'] += 1
    # (> 1 options) & (1 option == 2 option)
    else:
        # try overall skill based comparison
        vals = list(map(lambda x: user2skill[x], users))
        table = pd.DataFrame({'user': users, 'votes': votes, 'skill': vals})
        agg = table.groupby('votes').agg(
            sum_skill=pd.NamedAgg(column='skill', aggfunc='sum'),
            sum_votes=pd.NamedAgg(column='user', aggfunc='count')
        ).sort_values(by=['sum_votes', 'sum_skill'], ascending=False)
        # check there is a leader by skills
        if agg['sum_skill'].iloc[0] > agg['sum_skill'].iloc[1]:
            res = agg.index[0]
            stats['skill_based'] += 1
        else:
            # top-3 answers by overall skills
            vals = list(map(lambda x: user2skill[x], users))
            table = pd.DataFrame({'user': users, 'votes': votes, 'skill': vals})
            table = table.sort_values(by='skill', ascending=False)
            if len(table) >= 3:
                sub = table.iloc[:3]
            else:
                sub = table
            agg = sub.groupby('votes').agg(
                sum_skill=pd.NamedAgg(column='skill', aggfunc='sum'),
                sum_votes=pd.NamedAgg(column='user', aggfunc='count')
            ).sort_values(by=['sum_votes', 'sum_skill'], ascending=False)
            if agg['sum_skill'].iloc[0] != agg['sum_skill'].iloc[1]:
                res = agg.index[0]
                stats['major_based'] += 1
            
            else:
                vals = list(map(lambda x: user2alpha[x], users))
                table = pd.DataFrame({'user': users, 'votes': votes, 'skill': vals})
                agg = table.groupby('votes').agg(
                    sum_skill=pd.NamedAgg(column='skill', aggfunc='sum'),
                    sum_votes=pd.NamedAgg(column='user', aggfunc='count')
                ).sort_values(by=['sum_votes', 'sum_skill'], ascending=False)
                # check there is a leader by skills
                if agg['sum_skill'].iloc[0] != agg['sum_skill'].iloc[1]:
                    res = agg.index[0]
                    stats['em_based'] += 1
                else:
                    res = agg.index[0]
                    stats['rest'] += 1

    preds_full[keys[i]] = res

C:\Users\artem\AppData\Roaming\Python\Python311\site-packages\crowdkit\aggregation\classification\glad.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[labels] = 0
C:\Users\artem\AppData\Roaming\Python\Python311\site-packages\crowdkit\aggregation\classification\glad.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[labels] = 0
C:\Users\artem\AppData\Roaming\Python\Python311\site-packages\crowdkit\aggregation\classification\glad.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the re

In [11]:
stats

{'total_agreement': 347,
 'majority': 239,
 'skill_based': 11,
 'major_based': 0,
 'em_based': 3,
 'rest': 0}

In [12]:
preds_full_df = pd.concat([pd.DataFrame(preds_full.keys(), columns=['formula']), pd.DataFrame(preds_full.values(), columns=['lb'])], axis=1)

### Сопоставление разметки и ground truth

Забираем задания из датасета с правильными ответами.

In [13]:
res_df = pd.read_csv('full_with_info.tsv', sep='\t')

In [14]:
res_df = res_df.rename({'INPUT:formula': 'formula', 'target': 'lb'}, axis=1)

После скачивания с Толоки в текстах рушится форматирование, потому нельзя просто сделать join двух табличек. Нужно убрать все "лишнее" форматирование сразу из двух табличек, чтобы остались только тексты, пунктуация и пробелы

In [15]:
def format_text(text):
    text = (text.strip().replace('\n', ' ').replace('\t', ' ')
            .replace('\r', ' ').replace('  ', ' ').replace('  ', ' ')
            .replace('  ', ' '))
    return text

res_df['formula'] = res_df['formula'].apply(format_text)
preds_full_df['formula'] = preds_full_df['formula'].apply(format_text)

Делаем left join, чтобы соединить голосование и правильные метки для одних и тех же заданий.

In [16]:
new = res_df.merge(preds_full_df, on='formula', how='left')

In [17]:
new_valid = new[new['lb_y'].notna()].copy()
len(new_valid)

600

Ни одна строка не была утеряна.

In [18]:
new_valid.head(1)

,formula,lb_x,depth_level,length,lb_y
0,((((5 + 0) + ((-2) * 6)) + (((-8) * (-4)) + (6...,43,"[2, 2, 2]",2,43


### Подсчет метрики

In [19]:
new_valid['lb_y'] = new_valid['lb_y'].astype(int)

In [20]:
(new_valid['lb_x'] == new_valid['lb_y']).mean()

0.935

Это мог бы быть финальный этап, но в данные попали примеры, где ответом является не целое число, а вещественное (дробное). Учитывать такие примеры - нечестно, так как разметчики не могли ответить нецелым числом в силу специфики задания (было невозможно сохранить нецелый ответ). Уберем такие примеры.

In [21]:
new_valid['formula'].apply(lambda x: eval(x) % 1 != 0).sum()

116

В 116 из 600 объектах ответ нецелый. Убираем.

In [22]:
new_valid_filter = new_valid[new_valid['formula'].apply(lambda x: eval(x) % 1 == 0)]

In [23]:
(new_valid_filter['lb_x'] == new_valid_filter['lb_y']).mean()

0.9979338842975206

`Accuracy = 0.998`